In [1]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

In [2]:
import pickle

In [6]:
X = pd.read_pickle('X_train.pkl')
X.head()

In [8]:
y = pd.read_pickle('y_train.pkl')

In [9]:
X.head()

35     Chief Justice Roberts, President Carter, Pres...
13     Fellow-Citizens:     We have assembled to rep...
26     My friends, before I begin the expression of ...
30     Senator Hatfield, Mr. Chief Justice, Mr. Pres...
16     My Fellow-Citizens:     When we assembled her...
Name: 0, dtype: object

In [11]:
X_train_tokens = X.map(word_tokenize).values
X_train_tokens[:5]

array([list(['Chief', 'Justice', 'Roberts', ',', 'President', 'Carter', ',', 'President', 'Clinton', ',', 'President', 'Bush', ',', 'President', 'Obama', ',', 'fellow', 'Americans', ',', 'and', 'people', 'of', 'the', 'world', ':', 'thank', 'you', '.', 'We', ',', 'the', 'citizens', 'of', 'America', ',', 'are', 'now', 'joined', 'in', 'a', 'great', 'national', 'effort', 'to', 'rebuild', 'our', 'country', 'and', 'to', 'restore', 'its', 'promise', 'for', 'all', 'of', 'our', 'people', '.', 'Together', ',', 'we', 'will', 'determine', 'the', 'course', 'of', 'America', 'and', 'the', 'world', 'for', 'years', 'to', 'come', '.', 'We', 'will', 'face', 'challenges', '.', 'We', 'will', 'confront', 'hardships', '.', 'But', 'we', 'will', 'get', 'the', 'job', 'done', '.', 'Every', 'four', 'years', ',', 'we', 'gather', 'on', 'these', 'steps', 'to', 'carry', 'out', 'the', 'orderly', 'and', 'peaceful', 'transfer', 'of', 'power', ',', 'and', 'we', 'are', 'grateful', 'to', 'President', 'Obama', 'and', 'First

In [13]:
from nltk.corpus import stopwords

In [16]:
import string

In [17]:
stop_words = stopwords.words('english') + list(string.punctuation) + ['-','--',"'s"]

In [21]:
stop_words[:5]

['i', 'me', 'my', 'myself', 'we']

In [28]:
clean_X_train = []
for speech in X_train_tokens:
    clean_X_train.append([token.lower() for token in speech if token not in stop_words])
clean_X_train[:]

[['chief',
  'justice',
  'roberts',
  'president',
  'carter',
  'president',
  'clinton',
  'president',
  'bush',
  'president',
  'obama',
  'fellow',
  'americans',
  'people',
  'world',
  'thank',
  'we',
  'citizens',
  'america',
  'joined',
  'great',
  'national',
  'effort',
  'rebuild',
  'country',
  'restore',
  'promise',
  'people',
  'together',
  'determine',
  'course',
  'america',
  'world',
  'years',
  'come',
  'we',
  'face',
  'challenges',
  'we',
  'confront',
  'hardships',
  'but',
  'get',
  'job',
  'done',
  'every',
  'four',
  'years',
  'gather',
  'steps',
  'carry',
  'orderly',
  'peaceful',
  'transfer',
  'power',
  'grateful',
  'president',
  'obama',
  'first',
  'lady',
  'michelle',
  'obama',
  'gracious',
  'aid',
  'throughout',
  'transition',
  'they',
  'magnificent',
  'today',
  '’',
  'ceremony',
  'however',
  'special',
  'meaning',
  'because',
  'today',
  'merely',
  'transferring',
  'power',
  'one',
  'administration',
  '

In [29]:
len(clean_X_train)

36

In [23]:
X_train_tokens

array([list(['Chief', 'Justice', 'Roberts', ',', 'President', 'Carter', ',', 'President', 'Clinton', ',', 'President', 'Bush', ',', 'President', 'Obama', ',', 'fellow', 'Americans', ',', 'and', 'people', 'of', 'the', 'world', ':', 'thank', 'you', '.', 'We', ',', 'the', 'citizens', 'of', 'America', ',', 'are', 'now', 'joined', 'in', 'a', 'great', 'national', 'effort', 'to', 'rebuild', 'our', 'country', 'and', 'to', 'restore', 'its', 'promise', 'for', 'all', 'of', 'our', 'people', '.', 'Together', ',', 'we', 'will', 'determine', 'the', 'course', 'of', 'America', 'and', 'the', 'world', 'for', 'years', 'to', 'come', '.', 'We', 'will', 'face', 'challenges', '.', 'We', 'will', 'confront', 'hardships', '.', 'But', 'we', 'will', 'get', 'the', 'job', 'done', '.', 'Every', 'four', 'years', ',', 'we', 'gather', 'on', 'these', 'steps', 'to', 'carry', 'out', 'the', 'orderly', 'and', 'peaceful', 'transfer', 'of', 'power', ',', 'and', 'we', 'are', 'grateful', 'to', 'President', 'Obama', 'and', 'First

In [32]:
X_train_tokens[:1]

array([list(['Chief', 'Justice', 'Roberts', ',', 'President', 'Carter', ',', 'President', 'Clinton', ',', 'President', 'Bush', ',', 'President', 'Obama', ',', 'fellow', 'Americans', ',', 'and', 'people', 'of', 'the', 'world', ':', 'thank', 'you', '.', 'We', ',', 'the', 'citizens', 'of', 'America', ',', 'are', 'now', 'joined', 'in', 'a', 'great', 'national', 'effort', 'to', 'rebuild', 'our', 'country', 'and', 'to', 'restore', 'its', 'promise', 'for', 'all', 'of', 'our', 'people', '.', 'Together', ',', 'we', 'will', 'determine', 'the', 'course', 'of', 'America', 'and', 'the', 'world', 'for', 'years', 'to', 'come', '.', 'We', 'will', 'face', 'challenges', '.', 'We', 'will', 'confront', 'hardships', '.', 'But', 'we', 'will', 'get', 'the', 'job', 'done', '.', 'Every', 'four', 'years', ',', 'we', 'gather', 'on', 'these', 'steps', 'to', 'carry', 'out', 'the', 'orderly', 'and', 'peaceful', 'transfer', 'of', 'power', ',', 'and', 'we', 'are', 'grateful', 'to', 'President', 'Obama', 'and', 'First

In [33]:
token_set =[]
for speech in X_train_tokens:
    token_set.extend(speech)
len(set(token_set))

8656

In [34]:
X_train_tokens[:1]

array([list(['Chief', 'Justice', 'Roberts', ',', 'President', 'Carter', ',', 'President', 'Clinton', ',', 'President', 'Bush', ',', 'President', 'Obama', ',', 'fellow', 'Americans', ',', 'and', 'people', 'of', 'the', 'world', ':', 'thank', 'you', '.', 'We', ',', 'the', 'citizens', 'of', 'America', ',', 'are', 'now', 'joined', 'in', 'a', 'great', 'national', 'effort', 'to', 'rebuild', 'our', 'country', 'and', 'to', 'restore', 'its', 'promise', 'for', 'all', 'of', 'our', 'people', '.', 'Together', ',', 'we', 'will', 'determine', 'the', 'course', 'of', 'America', 'and', 'the', 'world', 'for', 'years', 'to', 'come', '.', 'We', 'will', 'face', 'challenges', '.', 'We', 'will', 'confront', 'hardships', '.', 'But', 'we', 'will', 'get', 'the', 'job', 'done', '.', 'Every', 'four', 'years', ',', 'we', 'gather', 'on', 'these', 'steps', 'to', 'carry', 'out', 'the', 'orderly', 'and', 'peaceful', 'transfer', 'of', 'power', ',', 'and', 'we', 'are', 'grateful', 'to', 'President', 'Obama', 'and', 'First

In [84]:
X_test = pd.read_pickle('X_test.pkl')

In [85]:
list_tokenized_headlines = tokenizer.texts_to_sequences(X_test)
X_test_vectors = sequence.pad_sequences(list_tokenized_headlines, maxlen=100)
X_test_vectors[:1]

array([[  17,   37,   15,   35, 2509,   14,    1,  490,   16, 2806,   39,
         215,   24,   56,  218,   36,  181,  624,   28,   19,  104,  346,
          57,    4,    1, 1888, 2184,    2,    1,  185,  294,    5, 1343,
           8,  297,  127,   27,   35, 3844,    4,  420,    1,  100,   29,
          18, 2137,   13,    5,  548, 1117,    3,   13, 7243,   18, 2500,
          32,    6,  280,    2,   26,   13,    1,  238,    2,   25,   72,
           3,    1, 1237,    2,   25,  281,   41,   76,  879, 1488,   51,
          11,  563, 6464,    5,    1, 1404,  733,    1, 5621, 4227,    3,
           1,  418,  432,   32,   16,    1,  278,    2,   21,   26,   46,
         706]])

In [35]:
tokenizer = text.Tokenizer(num_words=8656)
tokenizer.fit_on_texts(list(X))
list_tokenized_headlines = tokenizer.texts_to_sequences(X)
X_t = sequence.pad_sequences(list_tokenized_headlines, maxlen=100)

In [36]:
X_t[:1]

array([[  41,    4,   23,  221,    5,   59, 1144, 1721,    3,  167,  428,
           3,  416,   30, 3126,    4, 3126,    3,   30,  855,    4,  855,
        2054,   58,  381,   74,   22,  107,   11, 4343,  248,  126,  600,
         126,  362,    3,  126,  775,   22, 1482,    7,  100,  452,    3,
         126,  454,    3, 1722,    3,  314,   22,  943, 1148,   39, 1483,
           1,  172,  215,    9,   22,   97,   91,  269,  248,    9,   22,
          97,   91, 2448,  248,    9,   22,   97,   91,  944,  248,    9,
          22,   97,   91,  774,  248,    3, 1484,  215,    9,   22,   97,
          91,   47,  248,  629,   74,  204,  778,   74,    3,  204,  778,
          91]])

In [38]:
y.head()

35    R
13    R
26    R
30    R
16    R
Name: 1, dtype: object

In [39]:
y.value_counts()

R     15
D     14
DR     4
F      2
W      1
Name: 1, dtype: int64

In [54]:
y_1 = []
for party in y:
    if party == 'R':
        y_1.append(0)
    if party == 'D':
        y_1.append(1)
    if party == 'DR':
        y_1.append(2)
    if party == 'F':
        y_1.append(3)
    if party == 'W':
        y_1.append(4) 
y_1 = np.array(y_1)

In [43]:
model = Sequential()
embedding_size = 128
model.add(Embedding(8656, embedding_size))
model.add(LSTM(25, return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [47]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [48]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         1107968   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 25)          15400     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 25)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                1300      
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                

In [61]:
model.fit(X_t, y_1, epochs=10, batch_size=1, validation_split=0.2)

Epoch 1/10
28/28 [==============================] - 2s 63ms/step - loss: 0.3127 - accuracy: 0.8929 - val_loss: 0.6968 - val_accuracy: 0.6250
Epoch 2/10
28/28 [==============================] - 2s 65ms/step - loss: 0.2933 - accuracy: 0.8929 - val_loss: 0.5533 - val_accuracy: 0.6250
Epoch 3/10
28/28 [==============================] - 3s 106ms/step - loss: 0.2594 - accuracy: 0.8929 - val_loss: 1.0294 - val_accuracy: 0.6250
Epoch 4/10
28/28 [==============================] - 2s 61ms/step - loss: 0.2216 - accuracy: 0.9286 - val_loss: 0.8467 - val_accuracy: 0.6250
Epoch 5/10
28/28 [==============================] - 2s 67ms/step - loss: 0.2248 - accuracy: 0.8929 - val_loss: 0.9369 - val_accuracy: 0.6250
Epoch 6/10
28/28 [==============================] - 2s 67ms/step - loss: 0.3345 - accuracy: 0.8929 - val_loss: 0.9666 - val_accuracy: 0.6250
Epoch 7/10
28/28 [==============================] - 2s 68ms/step - loss: 0.2197 - accuracy: 0.9286 - val_loss: 0.4118 - val_accuracy: 0.7500
Epoch 8/10
2

In [60]:
model.save('RNN2.h5')

In [62]:
clean_X_train[:1]

[['chief',
  'justice',
  'roberts',
  'president',
  'carter',
  'president',
  'clinton',
  'president',
  'bush',
  'president',
  'obama',
  'fellow',
  'americans',
  'people',
  'world',
  'thank',
  'we',
  'citizens',
  'america',
  'joined',
  'great',
  'national',
  'effort',
  'rebuild',
  'country',
  'restore',
  'promise',
  'people',
  'together',
  'determine',
  'course',
  'america',
  'world',
  'years',
  'come',
  'we',
  'face',
  'challenges',
  'we',
  'confront',
  'hardships',
  'but',
  'get',
  'job',
  'done',
  'every',
  'four',
  'years',
  'gather',
  'steps',
  'carry',
  'orderly',
  'peaceful',
  'transfer',
  'power',
  'grateful',
  'president',
  'obama',
  'first',
  'lady',
  'michelle',
  'obama',
  'gracious',
  'aid',
  'throughout',
  'transition',
  'they',
  'magnificent',
  'today',
  '’',
  'ceremony',
  'however',
  'special',
  'meaning',
  'because',
  'today',
  'merely',
  'transferring',
  'power',
  'one',
  'administration',
  '

In [63]:
import nltk
from nltk.corpus import stopwords
import string
from nltk import word_tokenize, FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
np.random.seed(0)

In [65]:
vectorizer = TfidfVectorizer()

In [67]:
temp_train = []
for speech in clean_X_train:
    temp_train.append(' '.join(speech))
temp_train[:1]

['chief justice roberts president carter president clinton president bush president obama fellow americans people world thank we citizens america joined great national effort rebuild country restore promise people together determine course america world years come we face challenges we confront hardships but get job done every four years gather steps carry orderly peaceful transfer power grateful president obama first lady michelle obama gracious aid throughout transition they magnificent today ’ ceremony however special meaning because today merely transferring power one administration another one party another – transferring power washington d.c. giving back american people for long small group nation ’ capital reaped rewards government people borne cost washington flourished – people share wealth politicians prospered – jobs left factories closed the establishment protected citizens country their victories victories triumphs triumphs celebrated nation ’ capital little celebrate stru

In [68]:
train_vectors = vectorizer.fit_transform(temp_train)

In [69]:
train_vectors[:1]

<1x7881 sparse matrix of type '<class 'numpy.float64'>'
	with 483 stored elements in Compressed Sparse Row format>

In [71]:
train_vectors.shape

(36, 7881)

In [73]:
rf = RandomForestClassifier(n_estimators = 100)

In [74]:
rf.fit(train_vectors,y_1)

RandomForestClassifier()

In [77]:
from sklearn.model_selection import cross_val_score

In [80]:
results = cross_val_score(rf,train_vectors, y_1, cv = 2)

C:\Users\caell\new_Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(("The least populated class in y has only %d"


In [83]:
model = keras.models.load_model('RNN1.h5')

In [82]:
import keras

In [86]:
y_pred = model.predict(X_test_vectors)

In [88]:
pd.DataFrame(y_pred)

,0,1,2,3,4
0,0.978939,0.017210,0.002999,0.000724,0.000128
1,0.899342,0.092989,0.005757,0.001578,0.000334
2,0.972884,0.015535,0.008491,0.002473,0.000617
3,0.950366,0.040628,0.007095,0.001500,0.000412
4,0.943905,0.049718,0.004877,0.001231,0.000270
5,0.853788,0.071159,0.056146,0.011899,0.007008
6,0.972868,0.020812,0.004982,0.001037,0.000301
7,0.396434,0.550308,0.042353,0.006073,0.004831
8,0.929894,0.060111,0.007699,0.001778,0.000518
9,0.009332,0.988941,0.001234,0.000331,0.000162


In [89]:
y_pred_real = pd.DataFrame(['R','R','R','R','R','R','R','D','R','D','R','R','R','D','R','R','R','R','D','R','D','R'])

In [90]:
y_pred_real

,0
0,R
1,R
2,R
3,R
4,R
5,R
6,R
7,D
8,R
9,D


In [99]:
y_pred_real.to_pickle('jj_ad_predictions.pkl')

In [92]:
y_pred_real.dtypes

0    object
dtype: object

In [94]:
type(y_pred_real.iloc[0])

pandas.core.series.Series

In [95]:
y_pred_real

,0
0,R
1,R
2,R
3,R
4,R
5,R
6,R
7,D
8,R
9,D


In [97]:
y_pred_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       22 non-null     object
dtypes: object(1)
memory usage: 304.0+ bytes


In [102]:
y_pred_real = pd.Series(y_pred_real[0])

In [100]:
predictions_path = 'jj_ad_predictions.pkl'
with open(predictions_path, 'rb') as file:
    predictions = pickle.load(file)
    
assert len(predictions) == 22, 'Predictions do not have the correct shape.'

assert np.all([type(x) == str for x in predictions]), 'Each prediction should be a string datatype.'

print('Your predictions passed the tests!')

AssertionError: Each prediction should be a string datatype.